# <img style="float: left; padding-right: 10px; width: 45px" src="https://github.com/Harvard-IACS/2018-CS109A/blob/master/content/styles/iacs.png?raw=true"> CS-S109A Introduction to Data Science 

## Lecture 8: Trees and Forests

**Harvard University**<br>
**Summer 2020**<br>
**Instructors:** Kevin Rader<br>
**Authors:** Rahul Dave, David Sondak, Pavlos Protopapas, Chris Tanner, Eleni Kaxiras, Kevin Rader

---

In [ ]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

# Table of Contents 
<ol start="0">
<li> Learning Goals </li>
<li> Decision Trees </li> 
<li> Bootstrap-Aggregated (Bagged) Trees </li> 
<li> Random Forests </li> 
   

    

## Learning Goals

This Jupyter notebook accompanies Lecture 8. By the end of this lecture, you should be able to:

- Be able to fit and visualize (through predictions) decision trees, bootstrap-aggregated (bagged) trees, and random forests. 
- Have a general sense of the difference between the 3 tree based methods.
- Tune the hyperparameters of a random forest.


In [ ]:
%matplotlib inline
import sys
import numpy as np
import pylab as pl
import pandas as pd
import sklearn as sk
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

from sklearn import tree
import sklearn.metrics as met

pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 20)
plt.rcParams["figure.figsize"] = (12,8)

## Part 0: Reading and Exploring the data 

We will be using the same data as lectures 3-5 (both the train and test splits): modeling `votergap` from the 2016 election (Trump-Clinton) from many predictors in the `county_election` dataset.

We start by reading in the datasets for you and visualizing the main predictors for now: `minority`:

**Important note: use the training dataset for all exploratory analysis and model fitting.  Only use the test dataset to evaluate and compare models.**

In [ ]:
elect_train = pd.read_csv("../data/county_election_train.csv")
elect_test = pd.read_csv("../data/county_election_test.csv")
elect_train.head()

In [ ]:
y_train = elect_train['votergap']
y_test = elect_test['votergap']

plt.hist(y_train)
elect_train.hist(column=['minority', 'density','obesity','female']);

**Q0.1** How would you describe these variables?  What issues does this create?  How can these issues be fixed?

*your answer here*

In [ ]:
print(elect_train.shape)
print(elect_test.shape)

In [ ]:
plt.plot(elect_train['minority'],y_train, '.');

In [ ]:
plt.plot(np.log(elect_train['minority']),y_train, '.');

**Q0.2** Which of the two versions of 'minority' would be a better choice to use as a predictor for inference?  For prediction?

*your answer here*

---

## Part 1: Decision Trees

We could use a simple Decision Tree regressor to predict votergap. That's not the aim of this lab, so we'll run a few of these models without any cross-validation or 'regularization' just to illustrate what is going on.

This is what you ought to keep in mind about decision trees.

from the docs:
```
max_depth : int or None, optional (default=None)
The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
min_samples_split : int, float, optional (default=2)
```

- The deeper the tree, the more prone you are to overfitting.
- The smaller `min_samples_split`, the more the overfitting. One may use `min_samples_leaf` instead. More samples per leaf, the higher the bias (aka, simpler, underfit model).

Below we fit 2 decision treees that limit the `max_depth`: a single split, and one with 2 splits (resulting in 4 leaves).

In [ ]:
from sklearn.tree import DecisionTreeRegressor

elect_train['logminority'] = np.log(elect_train['minority'])
elect_test['logminority'] = np.log(elect_test['minority'])

dummy_x = np.arange(np.min(elect_train['logminority']),np.max(elect_train['logminority']),0.01)

plt.plot(elect_train['logminority'],y_train,'.')

for i in [1,2]:
    dtree = DecisionTreeRegressor(max_depth=i)
    dtree.fit(elect_train[['logminority']],y_train)
    plt.plot(dummy_x , dtree.predict(dummy_x.reshape(-1,1)), label=("max depth ="+str(i)), alpha=0.8, lw=4)

plt.legend();

And the actual decision tree can be *printed out* using [sklearn.tree.plot_tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html):

In [ ]:
from sklearn import tree

tree.plot_tree(dtree, filled=True)
plt.show()

**Q1.1** Interpret the printed out tree above: how does it match the scatterplot visualization of the regression tree?

*your answer here*

**Q1.2** Play around with the various arguments to define the complexity of the decision tree: `max_depth`,`min_samples_split`, and `min_samples_leaf` (do 1 at a time for now, you can use multiple of these arguments).  Roughly, at what point do these start to overfit?

In [ ]:
######
# your code here
######

**Q1.3** Perform 5-fold cross-validation to determine what the best `max_depth` would be for a single regression tree using the entire `X_train` feature set defined below.  Visualize the results with mean +/- 2 sd's across the validation sets.  Interpret the result.

In [ ]:
from sklearn.model_selection import cross_val_score

elect_train['logdensity'] = np.log(elect_train['density'])
elect_train['loghispanic'] = np.log(elect_train['hispanic'])

elect_test['logdensity'] = np.log(elect_test['density'])
elect_test['loghispanic'] = np.log(elect_test['hispanic'])

X_train = elect_train[['logminority', 'logdensity','loghispanic','obesity','female','income','bachelor','inactivity']]
X_test = elect_test[['logminority', 'logdensity','loghispanic','obesity','female','income','bachelor','inactivity']]


depths = list(range(1, 21))
train_scores = []
cvmeans = []
cvstds = []
cv_scores = []
for depth in depths:
    dtree = DecisionTreeRegressor(max_depth=depth)
    ######
    # your code here: 
    # Perform 5-fold cross validation and store results
    ######
    

#cvmeans = np.array(cvmeans)
#cvstds = np.array(cvstds)

In [ ]:
######
# your code here: 
# plot means and shade the 2 SD interval
######
# hint: use 'plt.fill_between' (with 'alpha' around 0.3) to plot the +/- 2sd's of CV scores behind the means.

#plt.legend()
#plt.ylabel("Accuracy")
#plt.xlabel("Max Depth")
#plt.xticks(depths);

Based on the plot above, it's clear that the training $R^2$ increases towards one as `max_depth` increases, while the test set's $R^2$ maxes out around 50% around a `max_depth` of 7. Any of those values would be reasonable to choose for a best predictive model.  To prevent overfitting and for parsimony, I would choose a `max_depth` of 7.  

Ok with this discussion in mind, lets improve this model by Bagging.

## Part 2: Bootstrap-Aggregating (called Bagging)

Whats the basic idea?

- A Single Decision tree is likely to overfit and is very 'jumpy' (discrete step-function).
- So lets introduce replication through Bootstrap sampling.
- **Bagging** uses bootstrap resampling to create different training datasets. This way each training will give us a different tree.
- Added bonus: the left off points can be used to as a natural "validation" set, so no need to 
- Since we have many trees that we will **average over for prediction**, we can choose a large `max_depth` and we are ok as we will rely on the law of large numbers to shrink this large variance, low bias approach for each individual tree.

In [ ]:
from sklearn.utils import resample

ntrees = 200
estimators = []
R2s = []
yhats_test = np.zeros((X_test.shape[0], ntrees))

plt.plot(elect_train['logminority'],y_train,'.')

for i in range(ntrees):
    simpletree = DecisionTreeRegressor(max_depth=5)
    boot_x, boot_y = resample(X_train[['logminority']], y_train)
    estimators = np.append(estimators,simpletree.fit(boot_x, boot_y))
    R2s = np.append(R2s,simpletree.score(X_test[['logminority']], y_test))
    yhats_test[:,i] = simpletree.predict(X_test[['logminority']])
    plt.plot(dummy_x, simpletree.predict(dummy_x.reshape(-1,1)), 'red', alpha=0.05)

**Q2.1** Interpret the plot above: how does bagging improve a single decision tree?  How could the resulting models be used to perform prediction?  Should the base trees be underfit, overfit, or just right?

*your answer here*

**Q2.2** Do lots of work here:
1. Edit the code below (which is just copied from above) to refit many bagged trees on the entire `X_train` feature set (without the plot...lots of predictors now so difficult to plot). 
2. Summarize how each of the separate trees performed (both numerically and visually) using $R^2$ as the metric.  How do they perform on average?
3. Combine the trees into one prediction and evaluate it using $R^2$.
4. Briefly discuss the results.  How will the results above change if 'max_depth' is increased?  What if it is decreased?

In [1]:
from sklearn.metrics import r2_score

######
# edit the code below
######


ntrees = 200
estimators = []
R2s = []
y_hats_test = np.zeros((X_test.shape[0], ntrees))

for i in range(ntrees):
    simpletree = DecisionTreeRegressor(max_depth=57)
    boot_x, boot_y = resample(X_train[['logminority']], y_train)
    estimators = np.append(estimators,simpletree.fit(boot_x, boot_y))
    R2s = np.append(R2s,simpletree.score(X_test[['logminority']], y_test))
    yhats_test[:,i] = simpletree.predict(X_test[['logminority']])


######
# your code here: 
# let's look at how the bagging does
######



NameError: name 'np' is not defined

*your answer here*





## Part 3: Random Forests

What's the basic idea?

Bagging alone is not enough randomization, because even after bootstrapping, we are mainly training on the same data points using the same variables, and will retain much of the overfitting.

So we will build each tree by splitting on "random" subset of predictors at each split (hence, each is a 'random tree').  This can't be done in with just one predcitor, but with more predictors we can choose what predictors to split on randomly and how many to do this on.  Then we combine many 'random trees' together by averaging their predictions, and this gets us a forest of random trees: a **random forest**.

Here are the parameters we can play with:

```
max_features : int, float, string or None, optional (default=”auto”)
- The number of features to consider when looking for the best split.
```

- `max_features`: Default splits on all the features and is probably prone to overfitting. You'll want to validate on this. 
- You can "validate" on the trees `n_estimators` as well but many times you will just look for the plateau in the trees as seen below.
- From decision trees you get the `max_depth`, `min_samples_split`, and `min_samples_leaf` as well, but you can leave those at defaults to get a maximally expanded tree.

Let's start by fitting a single attempt at a random forest, with `n_estimators` = 50, `max_features` = 0.33, and `max_depth` = 20:

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf1 = RandomForestRegressor(oob_score=True, n_estimators=100, max_features=0.33, max_depth=20)
rf1.fit(X_train, y_train)


**Q3.1** What is the predicted votergap for 'Middlesex County' in 'Massachusetts' (in the training set) for this random forest model?  Determine $R^2$ in the test set for this random forest model.  

In [ ]:

#this will be helpful
X_train[(elect_train['state']=='Massachusetts') & (elect_train['county']=='Middlesex County')]

######
# your code here
######


**Q3.2** Play around with the 3 hyperparameters to try to improve the predictive ability of the random forest (call it `rf2`).  How much has this improved the $R^2$ in the test set?  What parameters did you settle on?  

In [ ]:
######
# your code here
######


*your answer here*

Let's automate this process: below we create a hyper-param Grid. We are preparing to use the bootstrap points not used in training for validation rather than having to 'lose observations' in train by doing standard cross-validation.


In [ ]:
# code from 
# Adventures in scikit-learn's Random Forest by Gregory Saunders
from itertools import product
from collections import OrderedDict
param_dict = OrderedDict(
    n_estimators = [100, 300, 500],
    max_features = [0.4, 0.5, 0.6, 0.7, 0.8]
)

param_dict.values()

### Using the OOB score.

We have been putting "validate" in quotes. This is because the bootstrap gives us left-over points! So we'll now engage in our very own version of a grid-search, done over the out-of-bag scores that `sklearn` gives us for free

In [ ]:
from itertools import product

In [ ]:
# make sure y_train is the correct data type...in case you have warnings
# print(y_train.shape,X_train.shape)
# y_train = np.ravel(y_train)
# note: n_jobs=-1` parallelizes the process to use all your CPU threads

#Let's Cross-val. on the two 'hyperparameters' we based our grid on earlier
results = {}
estimators= {}
for ntrees, maxf in product(*param_dict.values()):
    params = (ntrees, maxf)
    est = RandomForestRegressor(oob_score=True, 
                                n_estimators=ntrees, max_features=maxf, max_depth=50, n_jobs=-1)
    est.fit(X_train, y_train)
    results[params] = est.oob_score_
    estimators[params] = est
outparams = max(results, key = results.get)
outparams

In [ ]:
rf_best = estimators[outparams]

In [ ]:
results

In [ ]:
rf_best.score(X_test, y_test)

Finally you can find the **feature importance** of each predictor in this random forest model. Whenever a feature is used in a tree in the forest, the algorithm will log the decrease in the splitting criterion (such as gini). This is accumulated over all trees and reported in `est.feature_importances_`

In [ ]:
pd.Series(rf_best.feature_importances_,index=list(X_train)).sort_values().plot(kind="barh");

Since our response isn't very symmetric, we may want to suppress outliers by using the `mean_absolute_error` instead. 

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, rf_best.predict(X_test))

**Q3.3** Tune the 'RandomForestRegressor' above to minimize 'mean_absolute_error' instead of the default ____________

*Note: `sklearn` supports this (`criterion='mae'`), but does not have completely arbitrary loss functions for Random Forests.*


In [ ]:
######
# your code here
######

This model would be expected to perform better in the test set when measuring mean absolute error, but not as good at mean squared error (but this is not guaranteed).  Here, we do not cross-validate, so that might explain why mean absolute error is worse.

Note: instead of using oob scoring, we could do cross-validation (with GridSearchCV), and a cv of 3 will roughly be comparable (same approximate train-vs.-validation set sizes). But this will take much more time as you are doing the fit 3 times plus the bootstraps. So at least three times as long!

In [ ]:
param_dict2 = OrderedDict(
    n_estimators = [400,600,800],
    max_features = [0.2, 0.4, 0.6, 0.8]
)

In [ ]:
from sklearn.model_selection import GridSearchCV
est2 = RandomForestRegressor(oob_score=False)
gs = GridSearchCV(est2, param_grid = param_dict2, cv=3, n_jobs=-1)
gs.fit(X_train, y_train)


In [ ]:
rf2 = gs.best_estimator_
rf2

In [ ]:
gs.best_score_

**Q3.4** What are the 3 *hyperparameters* for a random forest (one of the hyperparameters come in many *flavors*)?  Which hyperparameters need to be tuned? 


*your answer here*



### Seeing error as a function of the proportion of predictors at each split

Let's look to see how `max_features` affects performance on the training set.

In [ ]:
# from http://scikit-learn.org/stable/auto_examples/ensemble/plot_ensemble_oob.html

feats = param_dict['max_features']
# 
error_rate = OrderedDict((label, []) for label in feats)

# Range of `n_estimators` values to explore.
min_estimators = 50
step_estimators = 50
num_steps = 6
max_estimators = min_estimators + step_estimators*num_steps
for label in feats:
    for i in range(min_estimators, max_estimators+1, step_estimators):
        clf = RandomForestRegressor(oob_score=True, max_features=label)
        clf.set_params(n_estimators=i)
        clf.fit(X_train, y_train)

        # Record the OOB error for each `n_estimators=i` setting.
        oob_error = 1 - clf.oob_score_
        error_rate[label].append((i, oob_error))

In [ ]:
print(feats)

In [ ]:
# Generate the "OOB error rate" vs. "n_estimators" plot.
for label, clf_err in error_rate.items():
    xs, ys = zip(*clf_err)
    plt.plot(xs, ys, label=label)

plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate")
plt.legend(loc="upper right")
plt.show()